# FMEA & Reliability Analysis (Updated)
This notebook computes RPN, fits Weibull distributions using MLE, plots per-mode Weibull fits, and shows mitigations.

In [ ]:
import pandas as pd
from pathlib import Path
p = Path('..') / 'data' / 'dataset.csv'
df = pd.read_csv(p)
df.head()

In [ ]:
# Compute RPN
f = df.copy()
f['RPN'] = f['Severity']*f['Occurrence']*f['Detection']
rpn = f.groupby('Failure_Mode').agg({'RPN':'mean','Time_to_Failure_hr':['mean','count']})
rpn.columns=['RPN_mean','TTF_mean','Count']
print(rpn.sort_values('RPN_mean', ascending=False))

In [ ]:
# Weibull MLE fits using scipy
try:
    from scipy.stats import weibull_min
    use_scipy = True
except Exception as e:
    use_scipy = False

weibull_results = {}
for mode, g in df.groupby('Failure_Mode'):
    t = g['Time_to_Failure_hr'].values
    if len(t) < 3:
        weibull_results[mode] = {'beta': None, 'eta': None, 'n': len(t)}
        continue
    if use_scipy:
        c, loc, scale = weibull_min.fit(t, floc=0)
        weibull_results[mode] = {'beta': float(c), 'eta': float(scale), 'n': len(t)}
    else:
        # fallback linearization
        t_sorted = np.sort(t)
        n = len(t_sorted)
        ranks = np.arange(1, n+1)
        F = (ranks - 0.3) / (n + 0.4)
        ln_t = np.log(t_sorted)
        y_lin = np.log(-np.log(1 - F))
        b, a = np.polyfit(ln_t, y_lin, 1)
        beta = float(b)
        eta = float(np.exp(-a / b))
        weibull_results[mode] = {'beta': beta, 'eta': eta, 'n': n}

import pandas as pd
pd.DataFrame.from_dict(weibull_results, orient='index').to_csv(Path('..') / 'plots' / 'weibull_parameters_mle_from_notebook.csv')
print('Saved weibull_parameters_mle_from_notebook.csv')

## View generated Weibull plots
The plots generated by the pipeline are saved in `plots/` as `weibull_plot_<Failure_Mode>.png`. Open them from the Jupyter file browser to inspect model fit quality.

## Mitigations
The project includes a fishbone with mitigations at `plots/fishbone_detailed_mitigations.png` and a machine-readable table at `plots/fishbone_mitigations_table.csv`.